# NBA BetIQ – Model Training & Evaluation

This notebook trains classification models to estimate win probabilities
for NBA bets, evaluates performance, and generates diagnostic plots.


In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    brier_score_loss,
    classification_report,
)
from sklearn.model_selection import GridSearchCV

from ml.visuals import (
    plot_probability_calibration,
    plot_roc_curve,
)

sns.set(style="whitegrid")


## 1. Load Processed Data


In [ ]:
X_train = pd.read_csv("../data/processed/X_train.csv")
X_test = pd.read_csv("../data/processed/X_test.csv")
y_train = pd.read_csv("../data/processed/y_train.csv").squeeze()
y_test = pd.read_csv("../data/processed/y_test.csv").squeeze()

X_train.head()


## 2. Baseline Logistic Regression

We start with a simple, interpretable baseline model.


In [ ]:
log_reg = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
    n_jobs=-1,
)

log_reg.fit(X_train, y_train)

y_train_proba_lr = log_reg.predict_proba(X_train)[:, 1]
y_test_proba_lr = log_reg.predict_proba(X_test)[:, 1]
y_test_pred_lr = (y_test_proba_lr >= 0.5).astype(int)


## 3. Baseline Evaluation


In [ ]:
def evaluate_model(name, y_true, y_proba, threshold=0.5):
    y_pred = (y_proba >= threshold).astype(int)
    acc = accuracy_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_proba)
    brier = brier_score_loss(y_true, y_proba)

    print(f"=== {name} ===")
    print(f"Accuracy: {acc:.3f}")
    print(f"AUC:      {auc:.3f}")
    print(f"Brier:    {brier:.3f}")
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))


evaluate_model("Logistic Regression (Test)", y_test, y_test_proba_lr)


## 4. Calibration and ROC Plots


In [ ]:
plot_probability_calibration(
    y_true=y_test,
    y_pred_proba=y_test_proba_lr,
    save_path="../visuals/prob_calibration_curve_log_reg.png"
)

plot_roc_curve(
    y_true=y_test,
    y_pred_proba=y_test_proba_lr,
    save_path="../visuals/roc_curve_log_reg.png"
)


## 5. Random Forest Model (Optional Boost)

We train a tree-based ensemble to capture non-linear relationships.


In [ ]:
rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    min_samples_split=5,
    min_samples_leaf=2,
    n_jobs=-1,
    random_state=42,
)

rf.fit(X_train, y_train)

y_test_proba_rf = rf.predict_proba(X_test)[:, 1]
evaluate_model("Random Forest (Test)", y_test, y_test_proba_rf)


## 6. Hyperparameter Tuning (Example)

Use grid search or randomized search for better hyperparameters.
This can be expanded later as needed.


In [ ]:
param_grid = {
    "n_estimators": [200, 400],
    "max_depth": [None, 10, 20],
}

rf_grid = GridSearchCV(
    estimator=RandomForestClassifier(
        min_samples_split=5,
        min_samples_leaf=2,
        n_jobs=-1,
        random_state=42,
    ),
    param_grid=param_grid,
    scoring="roc_auc",
    cv=3,
    n_jobs=-1,
    verbose=1,
)

rf_grid.fit(X_train, y_train)

print("Best params:", rf_grid.best_params_)
print("Best CV AUC:", rf_grid.best_score_)

best_rf = rf_grid.best_estimator_
y_test_proba_best = best_rf.predict_proba(X_test)[:, 1]
evaluate_model("Best RF (Test)", y_test, y_test_proba_best)


## 7. Save Final Model

Persist the chosen model for use in the FastAPI backend.


In [ ]:
import joblib
import os

os.makedirs("../ml/models", exist_ok=True)
joblib.dump(best_rf, "../ml/models/betiq_model.pkl")
joblib.dump(X_train.columns.tolist(), "../ml/models/feature_columns.pkl")


## 8. Feature Importance (Tree-Based Models Only)


In [ ]:
if hasattr(best_rf, "feature_importances_"):
    importances = pd.Series(best_rf.feature_importances_, index=X_train.columns)
    importances.sort_values(ascending=False).head(20).plot(kind="bar")
    plt.title("Top 20 Feature Importances – Best RF")
    plt.ylabel("Importance")
    plt.tight_layout()
    plt.show()


## 9. Summary

We trained baseline and ensemble models, evaluated performance, and saved
the final model and feature list for deployment.
